In [55]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from numpy import linalg as LA
import os
import urllib.request
import colorama
from colorama import Fore, Style
import csv
%matplotlib inline

In [56]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset,DataLoader

<font face="微软雅黑" size=5 color=#A52A2A >Task1: Data preparation and display

In [57]:
#dataset(index()+real_part(24)+imaginary_part(24)+q(4))
raw_data=pd.read_csv('MyData_25000.csv')
df=pd.DataFrame(raw_data)
df=df.set_index('No')
df.head(5)
#df.tail(5)

,Real(h)_1,Real(h)_2,Real(h)_3,Real(h)_4,Real(h)_5,Real(h)_6,Real(h)_7,Real(h)_8,Real(h)_9,Real(h)_10,...,Imag(h)_19,Imag(h)_20,Imag(h)_21,Imag(h)_22,Imag(h)_23,Imag(h)_24,q_1,q_2,q_3,q_4
No,,,,,,,,,,,,,,,,,,,,,
1,0.020829,-41.950315,11.304785,4.916935,6.917294,5.262758,-10.742547,6.272788,0.781039,48.216098,...,-0.310428,5.749396,-6.596871,36.595822,8.692891,8.919924,0.712941,0.004418,0.157381,0.125260
2,11.989581,-4.038581,0.574468,5.078650,17.227288,15.862075,-9.494032,-8.335850,-0.016623,-50.099146,...,5.483431,-2.406380,-7.550119,-19.652089,1.705091,-2.249488,0.113186,0.013086,0.396980,0.476748
3,-0.797186,1.626110,50.091571,22.412621,-4.083873,-1.277553,-56.697669,11.935711,1.645857,1.070870,...,0.121436,3.992097,2.150922,-8.683767,4.323549,-0.442214,0.100918,0.788928,0.008065,0.102089
4,20.523107,-4.617390,8.435654,1.237661,25.271087,6.760474,50.205191,5.272745,-10.572534,-16.605411,...,26.575939,2.634083,-0.018737,-0.160407,-69.141834,5.105702,0.091481,0.120804,0.005015,0.782699
5,6.746797,-7.205020,-8.552247,1.222384,4.219274,-5.546848,-4.124834,6.571589,1.335697,30.289894,...,26.650121,-2.186115,-0.969020,-16.074266,23.099630,1.324784,0.411339,0.013081,0.051570,0.524010


In [58]:
#supress warnings
import warnings 
from pandas.core.common import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)


In [59]:
#get h matrix(2*24) input and column q vector(4*1) output
def parse_data(df,N,K,form): 
    df_c=df.copy()
    for k in range(N*K): #0-23
        df_c['Real(h)_'+str(k+1)] = df_c['Real(h)_'+str(k+1)].astype(form)
        df_c['Imag(h)_'+str(k+1)] = df_c['Imag(h)_'+str(k+1)].astype(form)
        if k<=3: #0-3
            df_c['q_'+str(k+1)] = df_c['q_'+str(k+1)].astype(form)
        
    df_c['h_ip']=[[]] * df.shape[0]#add new columns  
    df_c['q_exp']=[[]] * df.shape[0]
    real=[]
    imag=[]
    q_col=[]
    for idx in range(len(df_c)):
        for i in range(N*K):
            real.append(df_c.iloc[idx]['Real(h)_'+str(i+1)])
            imag.append(df_c.iloc[idx]['Imag(h)_'+str(i+1)])
            if i<=3:
                q_col.append([df_c.iloc[idx]['q_'+str(i+1)]])      
            df_c.at[idx+1,'h_ip']=[real,imag]
            df_c.at[idx+1,'q_exp']=q_col
        real=[]
        imag=[]
        q_col=[]
    for t in range(N*K):
        df_c.drop('Real(h)_'+str(t+1),axis=1,inplace=True) #no need to keep after reshaping
        df_c.drop('Imag(h)_'+str(t+1),axis=1,inplace=True)
        if t<=3:
            df_c.drop('q_'+str(t+1),axis=1,inplace=True) 
    return df_c

In [60]:
#reshaped dataset partly display
N,K=6,4
form=float
df_c=parse_data(df,N,K,form).copy()
df_c.head(5)


,h_ip,q_exp
No,,
1,"[[0.0208288733174387, -41.9503148729969, 11.30...","[[0.7129409417896541], [0.00441774112262481], ..."
2,"[[11.9895810316306, -4.03858116245368, 0.57446...","[[0.11318635455657002], [0.0130859197889614], ..."
3,"[[-0.797185851656008, 1.62610997703591, 50.091...","[[0.100918407171242], [0.7889278873124929], [0..."
4,"[[20.5231067503686, -4.61738966807714, 8.43565...","[[0.0914813174107049], [0.120804203424745], [0..."
5,"[[6.74679662912277, -7.205020055273139, -8.552...","[[0.41133938547927296], [0.0130805982325663], ..."


In [61]:
#dataset search/display
data_index=2

h_info=df_c.iloc[data_index-1]['h_ip']
q_info=df_c.iloc[data_index-1]['q_exp']
print(Fore.BLUE +'h input shown as:\n'+Fore.BLACK+str(np.array(h_info))) #use array to show a clean format, originl data is nested lists
print(Fore.BLUE +'q expected shown as:\n'+Fore.BLACK+str(np.array(q_info)))
  

h input shown as:
[[ 1.19895810e+01 -4.03858116e+00  5.74468454e-01  5.07865042e+00
   1.72272878e+01  1.58620749e+01 -9.49403237e+00 -8.33585008e+00
  -1.66234905e-02 -5.00991461e+01  3.65507381e+00  4.08385945e+00
   6.32668040e+00 -2.62171380e+01 -4.46792980e-02  1.09319612e+01
   1.65333515e+01 -3.75703660e+01 -6.27894605e+00  4.31542211e-01
  -1.77479800e+01  3.08630702e+00  4.00718646e+00 -1.21261917e+00]
 [ 6.42273961e+00  1.56075949e+01  3.32134597e+00 -3.62151558e+00
  -1.25639474e+00  1.23407540e+01 -1.06952720e+01 -7.05975296e+00
  -8.56067556e+00  2.51848138e+01 -3.46552598e+00 -2.92814889e+00
   5.15426772e+00  6.52295796e+01 -9.14102594e+00 -1.04014101e+00
   5.45715955e+00 -3.82909227e+01  5.48343068e+00 -2.40638033e+00
  -7.55011867e+00 -1.96520888e+01  1.70509122e+00 -2.24948768e+00]]
q expected shown as:
[[0.11318635]
 [0.01308592]
 [0.3969799 ]
 [0.47674783]]


In [62]:
from sklearn.model_selection import train_test_split

In [63]:
#train/test split-->0.8(20000)/0.2(5000),25000 samples in total
h=df_c.drop(columns = ['q_exp']).copy()
q=df_c['q_exp']

train_ratio=0.8


h_train, h_test, q_train, q_test = train_test_split(h,q, train_size=train_ratio)


print(h_train.shape[0]), print(q_train.shape[0])
print(h_test.shape[0]), print(q_test.shape[0])


20000
20000
5000
5000


(None, None)

In [64]:
# load data to tensor for use
import torch.utils.data as data_utils
class Init_Data(Dataset):
    def __init__(self,h_dataset,q_dataset):
        h_data=h_dataset
        q_data=q_dataset
        self.len=h_data.shape[0]
        self.h_data=torch.tensor(np.stack(h_data['h_ip']).astype(np.float64)) 
        self.q_data=torch.tensor(np.stack(q_data).astype(np.float64)) 
    
    def __getitem__(self,index):
        return self.h_data[index],self.q_data[index]
    
    def __len__(self):
        return self.len

In [65]:
#check tensor process works or not
print(torch.tensor(np.stack(h_train['h_ip']).astype(np.float64)).size()),
print(torch.tensor(np.stack(q_train).astype(np.float64)).size())

torch.Size([20000, 2, 24])
torch.Size([20000, 4, 1])


In [66]:
#apply as tensordata and load into dataloader
train_data=Init_Data(h_train,q_train)
test_data=Init_Data(h_test,q_test)


train_loader=DataLoader(dataset=train_data,batch_size=200,
                          shuffle=True,num_workers=0)
test_loader=DataLoader(dataset=test_data,batch_size=200,
                         shuffle=False,num_workers=0)

#next(iter(train_loader)) #h_train+q_train

<font face="微软雅黑" size=5 color=#A52A2A >Task2: Module Completion

In [67]:
class BNN_P1(nn.Module):
    def __init__(self):
        super(BNN_P1,self).__init__()
        self.conv1=nn.Conv2d(1,8,kernel_size=3,stride=1,padding=1,bias=True)         #layer 2, convolutional 
        self.bn1=nn.BatchNorm2d(8,eps=1e-03, momentum=0.99)                          #layer 3,batch normalization
        self.conv2=nn.Conv2d(8,8,kernel_size=3,stride=1,padding=1,bias=True)        #layer 5,convolutional
        self.bn2=nn.BatchNorm2d(8,eps=1e-03, momentum=0.99)                         #layer 7,batch normalization
        self.fc1=nn.Linear(384,4)                                                   #layer 9,fully connected
        self.sigmoid=nn.Sigmoid()                                                  #layer 10,sigmoid
        
    def forward(self,x):
        batch_size=x.size(0)
        x=self.conv1(x)
#         print(self.conv1.weight.size())           #torch.Size([8, 1, 3, 3])
#        print(self.conv1.bias.size())              #torch.Size([8])
#         print(x.shape)                         #torch.Size([200, 8, 2, 24])
        x=self.bn1(x)
#         print(x.shape)                         #torch.Size([200, 8, 2, 24])
        x=x=F.relu(x)                         #layer 4, ReLU activation
#         print(x.shape)                         #torch.Size([200, 8, 2, 24])
        x=self.conv2(x)
#         print(self.conv2.weight.size())           #torch.Size([8, 8, 3, 3])
#         print(self.conv2.bias.size())            #torch.Size([8])
#         print(x.shape)                         #torch.Size([200, 8, 2, 24])
        x=self.bn2(x)
#         print(x.shape)                         #torch.Size([200, 8, 2, 24])
        x=x=F.relu(x)                         #layer 6, ReLU activation
#         print(x.shape)                         #torch.Size([200, 8, 2, 24])
        x=x.view(batch_size,-1)               #layer 8,flatten 
#         print(x.shape)                        #torch.Size([200, 384])
        x=self.fc1(x)
#         print(x.shape)                        #torch.Size([200, 4])
        x=self.sigmoid(x)            
        x=x.unsqueeze(-1)                     #output layer, 4 feature to [4,1]
#         print(x.shape)                        #torch.Size([200, 4, 1])
        
        
        return x
    

In [68]:
 #Glorot normal inintializer for weights and bias initialized to 0
def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)
    elif isinstance(m, nn.Conv2d):
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)
        

<font face="微软雅黑" size=5 color=#A52A2A >Task3: Main

In [69]:
#model structure display
model=BNN_P1()
model=model.double()
model.apply(init_weights)

BNN_P1(
  (conv1): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(8, eps=0.001, momentum=0.99, affine=True, track_running_stats=True)
  (conv2): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(8, eps=0.001, momentum=0.99, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=384, out_features=4, bias=True)
  (sigmoid): Sigmoid()
)

In [70]:
criterion=torch.nn.MSELoss() #default reduction='Mean' 
optimizer=optim.Adam(model.parameters(),lr=0.001)

In [71]:
def train(epoch):
    running_loss=0.0
    for batch_idx,data in enumerate(train_loader,0):
        inputs,target=data
        optimizer.zero_grad()
        #forward
        inputs =inputs.unsqueeze(1) #add one dimension(channel)
        q_pred=model(inputs)
        #backward
        #target = target.unsqueeze(1)
        loss=criterion(q_pred,target)
#         print(target.size())
#         print(q_pred.size())
        loss.backward()
        #update
        optimizer.step()
        
        running_loss+=loss.item()
        if batch_idx%50==49: #100 iteration with batch size 200 by 20000 training samples, print loss every 50 iterations
            print('[%d,%9d] loss: %.9f'%(epoch+1,batch_idx+1,running_loss/200)) #mention the epoch index
            running_loss=0.0
   

In [82]:
def test_default_check(): #without traning, check what happened and the accuracy performance
    correct=0
    total=0
    check_point=0
    with torch.no_grad():
        for data in test_loader:
            original_input,labels=data
#             print(len(original_input))
#             print(len(labels))
#             print(original_input[1])
#             print(labels[1])
            ip3 =original_input.unsqueeze(1)
            outputs=model(ip3)
            predicted=outputs.data
            total+=labels.size(0)
            print(model.parameters())
#             print(predicted[1])
#             print(labels[1])
#             print(abs(LA.norm(predicted[1]-labels[1]))**2)
#             print((abs(LA.norm(predicted[1]-labels[1]))**2)<=0.01)
            
            for i in range(predicted.size(0)):
                if bool((abs(LA.norm(predicted[i]-labels[i]))**2)<=0.01): #error bearance
                    correct+=1
            check_point+=1
            print(36*'-'+'Check point '+str(check_point)+36*'-')
    
    print('Accuracy on test set: %d %% [%d/%d]'%(100*correct/total,correct,total))

In [83]:
def test():
    correct=0
    total=0
    with torch.no_grad():
        for data in test_loader:
            original_input,labels=data
            ip3 =original_input.unsqueeze(1)
            outputs=model(ip3)
            predicted=outputs.data
            total+=labels.size(0)
            for i in range(predicted.size(0)):
                if bool((abs(LA.norm(predicted[i]-labels[i]))**2)<=0.01):
                    correct+=1 
    present_accuracy=100*correct/total
    print('Accuracy on test set: %d %% [%d/%d]'%(present_accuracy,correct,total))

    torch.save(model.state_dict(), 'best-model-parameters.pt')
    torch.save(optimizer.state_dict(), 'best-optimizer-parameters.pt')
    print(Fore.RED+'Model parameters and optimizer saved this point with accuracy: %.3f %%' %(present_accuracy)+Fore.BLACK)    

In [84]:
if __name__=='__main__':
    epoch=100
    test_default_check() #without training   

<generator object Module.parameters at 0x0000015A86566F90>
------------------------------------Check point 1------------------------------------
<generator object Module.parameters at 0x0000015A86566900>
------------------------------------Check point 2------------------------------------
<generator object Module.parameters at 0x0000015A86566900>
------------------------------------Check point 3------------------------------------
<generator object Module.parameters at 0x0000015A86566900>
------------------------------------Check point 4------------------------------------
<generator object Module.parameters at 0x0000015A86566900>
------------------------------------Check point 5------------------------------------
<generator object Module.parameters at 0x0000015A86566900>
------------------------------------Check point 6------------------------------------
<generator object Module.parameters at 0x0000015A86566900>
------------------------------------Check point 7----------------------

In [75]:
for i in range(epoch):
        train(i)
        test()

[1,       50] loss: 0.018887539
[1,      100] loss: 0.012500345
Accuracy on test set: 1 % [55/5000]
Model parameters and optimizer saved this point with accuracy: 1.100 %
[2,       50] loss: 0.009257875
[2,      100] loss: 0.006884920
Accuracy on test set: 6 % [329/5000]
Model parameters and optimizer saved this point with accuracy: 6.580 %
[3,       50] loss: 0.004927477
[3,      100] loss: 0.003855766
Accuracy on test set: 14 % [730/5000]
Model parameters and optimizer saved this point with accuracy: 14.600 %
[4,       50] loss: 0.003265832
[4,      100] loss: 0.002882821
Accuracy on test set: 20 % [1000/5000]
Model parameters and optimizer saved this point with accuracy: 20.000 %
[5,       50] loss: 0.002616594
[5,      100] loss: 0.002524519
Accuracy on test set: 23 % [1156/5000]
Model parameters and optimizer saved this point with accuracy: 23.120 %
[6,       50] loss: 0.002321927
[6,      100] loss: 0.002287217
Accuracy on test set: 24 % [1202/5000]
Model parameters and optimizer

[45,       50] loss: 0.001463011
[45,      100] loss: 0.001507866
Accuracy on test set: 37 % [1857/5000]
Model parameters and optimizer saved this point with accuracy: 37.140 %
[46,       50] loss: 0.001463855
[46,      100] loss: 0.001501673
Accuracy on test set: 36 % [1821/5000]
Model parameters and optimizer saved this point with accuracy: 36.420 %
[47,       50] loss: 0.001496817
[47,      100] loss: 0.001476389
Accuracy on test set: 37 % [1885/5000]
Model parameters and optimizer saved this point with accuracy: 37.700 %
[48,       50] loss: 0.001483756
[48,      100] loss: 0.001485746
Accuracy on test set: 36 % [1848/5000]
Model parameters and optimizer saved this point with accuracy: 36.960 %
[49,       50] loss: 0.001463318
[49,      100] loss: 0.001495931
Accuracy on test set: 37 % [1879/5000]
Model parameters and optimizer saved this point with accuracy: 37.580 %
[50,       50] loss: 0.001461749
[50,      100] loss: 0.001495000
Accuracy on test set: 37 % [1854/5000]
Model para

[89,       50] loss: 0.001437650
[89,      100] loss: 0.001451812
Accuracy on test set: 37 % [1894/5000]
Model parameters and optimizer saved this point with accuracy: 37.880 %
[90,       50] loss: 0.001436186
[90,      100] loss: 0.001456150
Accuracy on test set: 38 % [1917/5000]
Model parameters and optimizer saved this point with accuracy: 38.340 %
[91,       50] loss: 0.001435798
[91,      100] loss: 0.001460238
Accuracy on test set: 37 % [1889/5000]
Model parameters and optimizer saved this point with accuracy: 37.780 %
[92,       50] loss: 0.001422390
[92,      100] loss: 0.001466750
Accuracy on test set: 37 % [1869/5000]
Model parameters and optimizer saved this point with accuracy: 37.380 %
[93,       50] loss: 0.001430274
[93,      100] loss: 0.001453099
Accuracy on test set: 37 % [1896/5000]
Model parameters and optimizer saved this point with accuracy: 37.920 %
[94,       50] loss: 0.001431053
[94,      100] loss: 0.001457579
Accuracy on test set: 37 % [1851/5000]
Model para

In [76]:
PATH1='best-model-parameters.pt'
model_best = BNN_P1()
model_best.load_state_dict(torch.load(PATH1))
model_best.eval()
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

PATH2='best-optimizer-parameters.pt'
optimizer_best = optim.Adam(model.parameters(),lr=0.001)
optimizer_best.load_state_dict(torch.load(PATH2))   
    
# Print optimizer's state_dict
print("Optimizer's state_dict:")
for var_name in optimizer_best.state_dict():
    print(var_name, "\t", optimizer_best.state_dict()[var_name])

Model's state_dict:
conv1.weight 	 torch.Size([8, 1, 3, 3])
conv1.bias 	 torch.Size([8])
bn1.weight 	 torch.Size([8])
bn1.bias 	 torch.Size([8])
bn1.running_mean 	 torch.Size([8])
bn1.running_var 	 torch.Size([8])
bn1.num_batches_tracked 	 torch.Size([])
conv2.weight 	 torch.Size([8, 8, 3, 3])
conv2.bias 	 torch.Size([8])
bn2.weight 	 torch.Size([8])
bn2.bias 	 torch.Size([8])
bn2.running_mean 	 torch.Size([8])
bn2.running_var 	 torch.Size([8])
bn2.num_batches_tracked 	 torch.Size([])
fc1.weight 	 torch.Size([4, 384])
fc1.bias 	 torch.Size([4])
Optimizer's state_dict:
state 	 {0: {'step': 10000, 'exp_avg': tensor([[[[ 4.9814e-04,  6.3316e-05,  6.5946e-05],
          [-6.7144e-05, -4.3240e-04, -1.1804e-03],
          [ 4.3024e-05,  6.0564e-06,  9.4250e-05]]],


        [[[ 1.6305e-05,  9.6281e-05, -1.0911e-04],
          [-6.4013e-04,  2.8457e-05,  1.2892e-03],
          [-1.3430e-04, -3.4323e-05,  1.0287e-04]]],


        [[[-1.8946e-04,  2.8487e-05,  2.0980e-04],
          [ 9.1549e-0

In [77]:
model_best=model_best.double()
for batch_idx,data in enumerate(test_loader,0):
        inputs_c,target_c=data
        inputs_c =inputs_c.unsqueeze(1) #add one dimension(channel)
        qq_pred=model_best(inputs_c)
print(qq_pred)

tensor([[[2.5442e-01],
         [4.4016e-04],
         [2.9115e-01],
         [5.0585e-01]],

        [[3.9218e-01],
         [1.6079e-02],
         [4.3464e-02],
         [2.9202e-01]],

        [[3.2774e-01],
         [4.5501e-01],
         [2.7436e-01],
         [2.6855e-02]],

        [[2.9853e-01],
         [1.5085e-01],
         [4.2238e-01],
         [2.1560e-01]],

        [[6.6012e-04],
         [3.3648e-02],
         [7.3067e-01],
         [2.9832e-01]],

        [[1.3166e-02],
         [7.6078e-01],
         [2.4259e-01],
         [3.9006e-02]],

        [[3.8646e-01],
         [4.6956e-02],
         [4.6325e-01],
         [6.7012e-02]],

        [[3.8266e-01],
         [7.1561e-01],
         [4.3650e-02],
         [4.9273e-04]],

        [[6.0753e-01],
         [5.2658e-02],
         [2.5016e-01],
         [7.7293e-02]],

        [[9.1838e-03],
         [2.5377e-01],
         [5.0475e-01],
         [6.5296e-02]],

        [[3.7235e-01],
         [3.9873e-01],
         [2.83

In [78]:
print(target_c)

tensor([[[2.8963e-01],
         [4.0694e-03],
         [3.1869e-01],
         [3.8761e-01]],

        [[6.3126e-01],
         [5.6597e-02],
         [6.3623e-02],
         [2.4852e-01]],

        [[3.0103e-01],
         [2.7704e-01],
         [3.8906e-01],
         [3.2867e-02]],

        [[2.4209e-01],
         [1.4489e-01],
         [4.0004e-01],
         [2.1298e-01]],

        [[7.7442e-03],
         [8.2846e-02],
         [6.3833e-01],
         [2.7108e-01]],

        [[3.6315e-02],
         [6.1753e-01],
         [2.5824e-01],
         [8.7912e-02]],

        [[2.9759e-01],
         [4.1544e-02],
         [5.4153e-01],
         [1.1933e-01]],

        [[2.8732e-01],
         [6.2618e-01],
         [8.2894e-02],
         [3.5977e-03]],

        [[5.9821e-01],
         [9.2714e-02],
         [1.7856e-01],
         [1.3052e-01]],

        [[3.6469e-02],
         [4.1320e-01],
         [4.8439e-01],
         [6.5945e-02]],

        [[3.5620e-01],
         [2.9775e-01],
         [1.30

In [32]:
print(abs(LA.norm(np.matrix(qq_pred.tolist()[0])-np.matrix(target_c.tolist()[0])))**2)

0.002867948432681673


In [25]:
print(np.array(inputs_c.tolist()[0]))

[[[ -0.51294741   0.6178873   51.62306908   5.44823032   3.65426827
    -3.01093356 -13.53950356  -1.56025288  -1.44079351   1.56296469
    24.87462745   2.95383685   2.61124672   1.3674396   34.13301594
   -11.55648713  -8.26357611   4.91939291 -53.27643637 -24.86528104
     4.00894988  -2.32117973  -2.06253676  -2.30004909]
  [  5.99095974  -4.49159199 -49.07075936 -21.89490308  -0.84235078
   -10.51984445 -17.81029306   5.54598769  -2.81276269  -2.72091415
    12.90401006   5.66574786  -2.6212928   -4.0328133  -31.22959827
    -6.26539648  -4.24248741  -2.43607834  -1.89063748  28.46253585
     2.43908376   2.6166561  -18.62812907  14.26094591]]]


In [26]:
h_list=[]
qpred_list=[]
qexp_list=[]
for i in range(200):
    h_list.append(inputs_c.tolist()[i])
    qexp_list.append(target_c.tolist()[i])
    qpred_list.append(qq_pred.tolist()[i])
data_200={'h_input': h_list, 'q_predict': qpred_list,'q_real':qexp_list} 
df_200=pd.DataFrame(data_200)  
df_200.index+=1
df_200.head(5)

,h_input,q_predict,q_real
1,"[[[-0.512947407036421, 0.6178872998767779, 51....","[[0.5244445961101116], [0.43266274246635256], ...","[[0.5046573489605529], [0.40291922279431297], ..."
2,"[[[-16.8117790363349, -10.0044179998228, -3.38...","[[0.00036290359642143706], [0.0936400726867578...","[[0.00560706454770033], [0.10915922223985002],..."
3,"[[[13.4900336351284, -2.60190054263718, -1.001...","[[0.06596011311037349], [0.5684434259091489], ...","[[0.0709008177199287], [0.5803498015790439], [..."
4,"[[[-1.7108286350794097, 9.39642148471085, 11.1...","[[0.2292788131518387], [0.16248023674493234], ...","[[0.204454114023573], [0.233062483490389], [0...."
5,"[[[2.54684634599772, 1.06977998064985, 0.23069...","[[0.049066321437711446], [0.4018047859637618],...","[[0.0619881544687333], [0.325993305188003], [0..."


In [27]:
print(np.array(df_200.iloc[1]['h_input']))

[[[ -16.81177904  -10.004418     -3.38962921    8.4221353    23.47718958
     -2.39255387    5.54909506    6.8309916   -81.21883192  -37.93005075
     -7.74364308    3.25948654   57.93467303  -19.89156712   12.10430475
     -6.90877518 -107.90977603  -24.50755992   12.03160419   -3.713654
   -127.62654664    8.13606902   16.7007179     7.72990122]
  [ -81.92182861   -3.60758854    2.00822605   -3.96455647  -70.03189398
      7.89107869   -3.56663283   -8.52139244   -5.42776396  -15.53090556
    -11.6021684     3.82845756  -32.00971682    9.0200967   -11.4143996
     -6.7129745   148.38339798   -2.24493977   -8.21423231   17.90405598
     72.87698679   15.24517797   -5.93196682    1.67728952]]]


In [28]:
df_200.to_csv('list_200_for_recovery.csv', encoding='utf-8')